In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import pymongo
import requests
import folium

In [2]:
# mongoDB에 저장된 구군_ALL 목록을 얻어온다.
mongoConnection = pymongo.MongoClient('mongodb://localhost:27017/')
starbucks = mongoConnection['starbucks']
gugunAll = starbucks['gugunAll']
gugunAllList = gugunAll.find({}, {'_id': 0, 'gugunAll_cd': 1, 'gugunAll_nm': 1})
# for s in gugunAllList:
    # print(s)

In [3]:
# mongoDB에 저장된 구군 목록을 얻어온다.
mongoConnection = pymongo.MongoClient('mongodb://localhost:27017/')
starbucks = mongoConnection['starbucks']
gugun = starbucks['gugun']
gugunList = gugun.find({}, {'_id': 0, 'gugun_cd': 1, 'gugun_nm': 1})
# for s in gugunAllList:
    # print(s)

In [4]:
sidoName = input('검색하려는 시도 이름을 입력하세요(예: 서울): ')
# sido 컬렉션에서 입력받은 시도 이름에 해당되는 도큐먼트 1건을 얻어온다.
sido_cd = gugunAll.find({'sido_nm': sidoName}, {'_id': 0, 'sido_cd': 1, 'sido_nm': 1}).next()
print(sido_cd['sido_cd'])
gugunList = gugunAll.find({'sido_nm': sidoName}, {'_id': 0, 'gugun_cd': 1, 'gugun_nm': 1})
for s in gugunList:
    print(s['gugun_nm'], end= ' ')

gugunName = input('검색하려는 구군 이름을 입력하세요(예: 강남구): ')
# sido 컬렉션에서 입력받은 시도 이름에 해당되는 도큐먼트 1건을 얻어온다.
gugun_cd = gugunAll.find({'gugun_nm': gugunName}, {'_id': 0, 'gugun_cd': 1, 'gugun_nm': 1}).next()
print(gugun_cd['gugun_cd'])

검색하려는 시도 이름을 입력하세요(예: 서울): 부산
05
강서구 금정구 기장군 남구 동구 동래구 부산진구 북구 사상구 사하구 서구 수영구 연제구 영도구 중구 해운대구 검색하려는 구군 이름을 입력하세요(예: 강남구): 해운대구
0516


In [5]:
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=3112E6S5DE'
req = requests.post(targetSite, data = {
    'ins_lat': 37.5717888, # 위도
    'ins_lng': 126.9661696, # 경도
    'p_sido_cd': sido_cd['sido_cd'], # 시도코드
    'p_gugun_cd':gugun_cd['gugun_cd'], # 구군코드
    'in_biz_cd': '', # ?
    'iend': 2000, # 서버가 응답하는 최대 매장의 개수
    'set_date': '' # ?
    # in_biz_cd, set_date를 넘겨주지 않으면 서버에서 데이터를 얻어오지 못한다.
})
storeList = req.json()
print(len(storeList['list']))

count = 0
for store in storeList['list']:
    count += 1
    print('{:4d}. {}: {}({}, {})'.format(count, store['s_name'], store['doro_address'], store['lat'], store['lot']))


32
   1. 그랜드조선 부산: 부산광역시 해운대구 해운대해변로 292 (중동)(35.1600338, 129.1630512)
   2. 센텀몰1F: 부산광역시 해운대구 센텀4로 15 (우동)(35.16981023, 129.12778297)
   3. 해운대중동역: 부산광역시 해운대구 좌동순환로 6 (중동)(35.1671846, 129.1678855)
   4. 센텀신세계3F: 부산광역시 해운대구 센텀남대로 35 (우동)(35.16906348, 129.12906568)
   5. 해운대 엑스더스카이: 부산광역시 해운대구 달맞이길30 (35.1599648, 129.1697564)
   6. 해운대좌동: 부산광역시 해운대구 좌동순환로 178 (좌동)(35.1775937, 129.1755895)
   7. 해운대NC: 부산광역시 해운대구 해운대로 813 (좌동)(35.1705843, 129.1770927)
   8. 월드마크센텀: 부산광역시 해운대구 센텀동로 25, 1층 (우동)(35.17137, 129.131112)
   9. 센텀KNN: 부산광역시 해운대구 센텀서로 30 (우동)(35.171765, 129.128934)
  10. 센텀드림월드: 부산광역시 해운대구 센텀2로 25, 센텀드림월드 1층 (우동)(35.167027, 129.132829)
  11. 해운대이마트: 부산광역시 해운대구 좌동순환로 511 (중동) 1층(35.166029, 129.167397)
  12. 해운대달맞이: 부산광역시 해운대구 달맞이길 189 (중동)(35.157925, 129.182307)
  13. 마린시티아이파크: 부산광역시 해운대구 마린시티2로 38 (우동)(35.1569194, 129.1434995)
  14. 제니스스퀘어: 부산광역시 해운대구 마린시티2로 33, 제니스스퀘어 (우동)(35.156657, 129.144091)
  15. 센텀시티역: 부산광역시 해운대구 센텀동로 9, 트럼프월드센텀아파트 1층 (우동)(35.169925, 129.132271)
  16. 블루스퀘

In [6]:
starbucksMap = folium.Map(location=[36.56009115, 127.83364774], zoom_start=7)

count = 0
for store in storeList['list']:
    count += 1
    popup = folium.Popup(store['s_name']+'점', max_width=300)
    folium.Marker(location=[store['lat'], store['lot']], popup=popup).add_to(starbucksMap)

starbucksMap